# Generating Counterfactuals

In this notebook, we will focus on generating counterfactuals from individual
datapoints. This will be implemented for the following models:

- Naive Bayes
- Fair Bayesian Network
- Fair Random Forest Classifier

By generating counterfactuals, we hope to gain insight into how the model uses
the different attributes in it's decisions.

In [15]:
import sys
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from forseti.bayesnet import latentLabelClassifier, interpretableNaiveBayes
import pandas as pd
from random import sample
import numpy as np
from forseti.datproc import translate_categorical

df = pd.read_csv("data/adult.csv")

clf = interpretableNaiveBayes()

tmp = df
label = "income"

clf.train(label, tmp, "NB")

In [23]:
test = df.drop(label, axis=1)
test, _ = translate_categorical(test)

In [ ]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem

class ElementwiseSphereWithConstraint(ElementwiseProblem):
    def _evaluate(self, x, out, *args, **kwargs):
        out["O1"] = clf.predict_probability(pd.DataFrame(test.iloc[0]).T).iloc[:, 1]

In [37]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_problem
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter

problem = get_problem("zdt1")

algorithm = NSGA2(pop_size=100)

res = minimize(problem,
               algorithm,
               ('n_gen', 200),
               seed=1,
               verbose=True)

plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(res.F, color="red")
plot.show()